In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
train_feature = pd.read_csv("./data/train_features.csv")
train_label = pd.read_csv("./data/train_labels.csv")
sample_submission = pd.read_csv("./data/sample_submission.csv")
test_feature = pd.read_csv("./data/test_features.csv")

In [44]:
train_feature

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,gy_acc_energy,acc_energy,gy_energy
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,9.632846,1.146962,12.465436
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,11.427630,1.200703,12.913284
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,12.447808,1.217403,13.725729
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,14.132423,1.209981,15.374021
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,14.906627,1.211254,16.074363
...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,22.523972,1.009050,25.963234
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,22.253594,1.002827,25.784692
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,21.747993,1.006239,25.628060
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,21.646303,1.001038,25.626266


In [7]:


train_feature['acc_energy'] = \
(train_feature['acc_x']**2 + train_feature['acc_y']**2 + train_feature['acc_z']**2)**(1/3)
test_feature['acc_energy'] = \
(test_feature['acc_x']**2 + test_feature['acc_y']**2 + test_feature['acc_z']**2)**(1/3)

train_feature['gy_energy'] =\
(train_feature['gy_x']**2 + train_feature['gy_y']**2 + train_feature['gy_z']**2)**(1/3)
test_feature['gy_energy'] = \
(test_feature['gy_x']**2 + test_feature['gy_y']**2 + test_feature['gy_z']**2)**(1/3)
train_feature['gy_acc_energy'] = \
((train_feature['gy_x'] - train_feature['acc_x'])**2 
 + (train_feature['gy_y'] - train_feature['acc_y'])**2 
 + (train_feature['gy_z'] - train_feature['acc_z']**2))**(1/3)

test_feature['gy_acc_energy'] = \
((test_feature['gy_x'] - test_feature['acc_x'])**2 
 + (test_feature['gy_y'] - test_feature['acc_y'])**2 
 + (test_feature['gy_z'] - test_feature['acc_z']**2))**(1/3)

In [58]:
x_train = []

for uid in tqdm(train_feature['id'].unique()):
    temp = np.array(train_feature[train_feature['id'] == uid].iloc[:,2:], np.float32).T
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:,np.newaxis]
print("------------------------------------")
print("x_train : ", x_train.shape)
print("------------------------------------")

x_test = []

for uid in tqdm(test_feature['id'].unique()):
    temp = np.array(test_feature[test_feature['id'] == uid].iloc[:,2:], np.float32).T
    x_test.append(temp)

x_test = np.array(x_test, np.float32)
x_test = x_test[:,:,:,np.newaxis]

print("------------------------------------")
print("x_test : ", x_test.shape)
print("------------------------------------")


  9%|███████                                                                         | 69/782 [00:00<00:01, 676.39it/s]

------------------------------------
x_train :  (3125, 9, 600, 1)
------------------------------------


100%|███████████████████████████████████████████████████████████████████████████████| 782/782 [00:01<00:00, 653.87it/s]

------------------------------------
x_test :  (782, 9, 600, 1)
------------------------------------


In [52]:
x_train

array([[[[ 1.20608664e+00],
         [ 1.28769648e+00],
         [ 1.30460918e+00],
         ...,
         [ 1.02564275e+00],
         [ 1.03155303e+00],
         [ 1.13815928e+00]],

        [[-1.79371431e-01],
         [-1.98973715e-01],
         [-1.95114493e-01],
         ...,
         [-2.27845505e-01],
         [-3.87862504e-01],
         [-4.26846206e-01]],

        [[-1.48446664e-01],
         [-1.82443753e-01],
         [-2.53382474e-01],
         ...,
         [-3.54516268e-01],
         [-2.77856708e-01],
         [-4.30263162e-01]],

        ...,

        [[ 9.63284683e+00],
         [ 1.14276304e+01],
         [ 1.24478083e+01],
         ...,
         [ 8.90397167e+00],
         [ 9.93936920e+00],
         [ 9.32347870e+00]],

        [[ 1.14696240e+00],
         [ 1.20070291e+00],
         [ 1.21740317e+00],
         ...,
         [ 1.07130718e+00],
         [ 1.08907735e+00],
         [ 1.18469703e+00]],

        [[ 1.24654360e+01],
         [ 1.29132833e+01],
         [

In [53]:
train_feature.shape

(1875000, 11)

In [60]:
x_train.shape

(3125, 9, 600, 1)

In [54]:
train_feature

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,gy_acc_energy,acc_energy,gy_energy
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,9.632846,1.146962,12.465436
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,11.427630,1.200703,12.913284
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,12.447808,1.217403,13.725729
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,14.132423,1.209981,15.374021
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,14.906627,1.211254,16.074363
...,...,...,...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437,22.523972,1.009050,25.963234
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087,22.253594,1.002827,25.784692
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125,21.747993,1.006239,25.628060
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478,21.646303,1.001038,25.626266


In [55]:
x_train = x_train.reshape(-1, 9)
df = pd.DataFrame(x_train, columns=['acc_x','acc_y','acc_z','gy_x','gy_y','gy_z','gy_acc_energy','acc_energy','gy_energy'])

In [61]:
df.shape

(1875000, 9)

In [50]:
df

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,gy_acc_energy,acc_energy,gy_energy
0,1.206087,1.287696,1.304609,1.293095,1.300887,1.289304,1.281405,1.242273,1.198871
1,1.113677,1.192241,1.230094,1.175210,1.170251,1.075277,0.993865,0.956588,0.977357
2,0.997987,0.914106,0.851735,0.790079,0.759773,0.736777,0.721108,0.702457,0.699058
3,0.623462,0.654510,0.682234,0.712764,0.729719,0.761901,0.676890,0.633556,0.628711
4,0.687481,0.689470,0.616131,0.659277,0.711882,0.696517,0.776035,0.855469,0.730303
...,...,...,...,...,...,...,...,...,...
1874995,14.794225,13.252866,12.103681,11.010658,10.420176,10.180671,9.353516,8.182703,6.983366
1874996,5.715691,3.864775,3.264111,6.661088,8.490607,9.846674,11.293477,12.677922,13.547388
1874997,14.968551,16.520185,17.245808,17.841192,18.683788,19.425898,20.225929,20.961702,21.503031
1874998,21.850443,22.274492,22.837259,23.358917,23.601198,23.951311,24.404491,24.605389,24.858891
